# Client Project: The Lab @ DC

## Project Title: {here}

### Authors: Kihoon Sohn, Brian Collins, Harsha Goonawardana, Priya Kakkar
- Cohorts of the Data Science Immersive, General Assembly @ Washington DC campus

In this notebook, we have Exploratory Data Analysis on the City Service Requests / ShotsSpotters datasets. **This is notebook 2 of 3.**

### Import Libraries

In [2]:
# import basic libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

### Read CSVs

In [3]:
csr_train   = pd.read_csv('./assets/csr/csr_train.csv', low_memory=False)
csr_test    = pd.read_csv('./assets/csr/csr_test.csv', low_memory=False)
shots_train = pd.read_csv('./assets/mpd/shots_train.csv', low_memory=False)
shots_test  = pd.read_csv('./assets/mpd/shots_test.csv', low_memory=False)

##### check null values and basic info on the datasets

In [4]:
csr_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1231233 entries, 0 to 1231232
Data columns (total 30 columns):
X                             1231233 non-null float64
Y                             1231233 non-null float64
OBJECTID                      1231233 non-null int64
SERVICECODE                   1231233 non-null object
SERVICECODEDESCRIPTION        1231233 non-null object
SERVICETYPECODEDESCRIPTION    1230379 non-null object
ORGANIZATIONACRONYM           1231232 non-null object
SERVICECALLCOUNT              1231233 non-null int64
ADDDATE                       1231233 non-null object
RESOLUTIONDATE                1145201 non-null object
SERVICEDUEDATE                1218530 non-null object
SERVICEORDERDATE              1231233 non-null object
INSPECTIONFLAG                1231233 non-null object
INSPECTIONDATE                434130 non-null object
INSPECTORNAME                 40361 non-null object
SERVICEORDERSTATUS            1230380 non-null object
STATUS_CODE               

In [5]:
csr_train.isnull().sum().sort_values(ascending=False)

INSPECTORNAME                 1190872
INSPECTIONDATE                 797103
DETAILS                        444566
MARADDRESSREPOSITORYID         189162
STATUS_CODE                    151801
RESOLUTIONDATE                  86032
CITY                            50324
STATE                           50324
STREETADDRESS                   49730
SERVICEDUEDATE                  12703
WARD                             6221
PRIORITY                         2677
SERVICETYPECODEDESCRIPTION        854
SERVICEORDERSTATUS                853
ZIPCODE                            16
ORGANIZATIONACRONYM                 1
SERVICEREQUESTID                    0
XCOORD                              0
INSPECTIONFLAG                      0
SERVICEORDERDATE                    0
YCOORD                              0
LATITUDE                            0
ADDDATE                             0
SERVICECALLCOUNT                    0
LONGITUDE                           0
SERVICECODEDESCRIPTION              0
SERVICECODE 

In [6]:
shots_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 1 columns):
version https://git-lfs.github.com/spec/v1    2 non-null object
dtypes: object(1)
memory usage: 96.0+ bytes


In [7]:
shots_train.isnull().sum().sort_values(ascending=False)

version https://git-lfs.github.com/spec/v1    0
dtype: int64

### Basic EDAs and Data Cleaning

##### Basic EDAs on Shots dataset

In [8]:
shots_train.head()

,version https://git-lfs.github.com/spec/v1
0,oid sha256:bb18af6c1c610635273f0447b6a58c09f6a...
1,size 2149381


In [9]:
shots_train.shape

(2, 1)

In [10]:
shots_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 1 columns):
version https://git-lfs.github.com/spec/v1    2 non-null object
dtypes: object(1)
memory usage: 96.0+ bytes


In [11]:
shots_train.isnull().sum().sort_values(ascending=False)

version https://git-lfs.github.com/spec/v1    0
dtype: int64

In [12]:
shots_train[shots_train.ID.isnull()]

AttributeError: 'DataFrame' object has no attribute 'ID'

In [31]:
def shot_spot_preprocess(df):
    shots_train.set_index(['ID'], inplace=True)
    shots_train.Source = shots_train.Source.apply(lambda DC: DC.replace('WashingtonDC', ''))
    shots_train.Date = pd.to_datetime(shots_train.Date, infer_datetime_format=True)
    return df

shots_train = shot_spot_preprocess(shots_train)

In [35]:
# shots_train.to_csv('./assets/mpd/shots_train_preprocessed.csv')

In [130]:
shots_train.Source.value_counts()

7D    10342
6D     8407
5D     3216
4D     2701
1D     1954
3D     1723
Name: Source, dtype: int64

In [47]:
shots_train.Type.value_counts()

Multiple_Gunshots         15858
Single_Gunshot            10034
Gunshot_or_Firecracker     2451
Name: Type, dtype: int64

##### Basic EDAs on City Service Requests Datasets

In [131]:
# Unnamed clearly the Original index column.  Delete that.
# X and Y seems to be closely related to 
# Is service code description the same as service type, just a little more information?
# Object ID can be set as the index.
csr_train.iloc[:5, :6]

,X,Y,OBJECTID,SERVICECODE,SERVICECODEDESCRIPTION,SERVICETYPECODEDESCRIPTION
0,-76.972735,38.897957,463232,S0011,Alley Cleaning,Street Cleaning
1,-76.991907,38.922865,463233,S0321,Recycling Collection - Missed,Recycling
2,-76.970891,38.874749,463234,S0031,Bulk Collection,Bulk Collection
3,-77.022678,38.942819,463235,S0311,Rat Abatement,DOH
4,-77.048840,38.898960,463236,S0276,Parking Meter Repair,TOA


In [132]:
# Service Type, Important to know the different values there.
# Organization to know who is on the task.
# Service call count to see how many times a call is needed.
# Everything just needs to be set to datetime.
csr_train.iloc[:5, 6:12]

,ORGANIZATIONACRONYM,SERVICECALLCOUNT,ADDDATE,RESOLUTIONDATE,SERVICEDUEDATE,SERVICEORDERDATE
0,DPW,1,2014-01-02T13:27:40.000Z,2014-01-15T07:43:42.000Z,2014-02-18T13:27:40.000Z,2014-01-02T13:27:40.000Z
1,DPW,1,2014-01-02T13:46:57.000Z,2014-01-06T12:39:39.000Z,2014-01-06T13:46:57.000Z,2014-01-02T13:46:57.000Z
2,DPW,1,2014-01-02T13:57:46.000Z,2014-01-14T14:29:16.000Z,2014-01-23T13:57:46.000Z,2014-01-02T13:57:46.000Z
3,DOH,1,2014-01-02T13:43:20.000Z,NaN,2014-02-24T13:43:20.000Z,2014-01-02T13:43:20.000Z
4,DDOT,1,2014-01-02T16:00:59.000Z,2014-01-07T16:33:48.000Z,2014-01-09T16:00:59.000Z,2014-01-02T16:00:59.000Z


In [133]:
# Service order data good.  Needs to be broken down into Datetime.
# Inspection Flag, what does that mean?
# Remove Inspector Name
# What does Status code contain? A lot of NaN could be bad
# Service Order Status, Important maybe?
csr_train.iloc[:5, 12:18]

,INSPECTIONFLAG,INSPECTIONDATE,INSPECTORNAME,SERVICEORDERSTATUS,STATUS_CODE,SERVICEREQUESTID
0,N,NaN,NaN,CLOSED,NaN,14-00000654
1,N,2014-01-06T12:39:00.000Z,"Bryant, Kevin",CLOSED,NaN,14-00000686
2,N,NaN,NaN,CLOSED,NaN,14-00000707
3,N,NaN,NaN,OPEN,NaN,14-00000677
4,N,NaN,NaN,CLOSED,NaN,14-00000877


In [134]:
# Service Request ID seems unimportant
# What is the XCOORD and YCOORD?
# Street Address can help us find our quandrants. Do we also want the address or is Latitude and Longitude
# Priority - how many unique values are in there?
csr_train.iloc[:5, 18:24]

,PRIORITY,STREETADDRESS,XCOORD,YCOORD,LATITUDE,LONGITUDE
0,STANDARD,2301 BENNING ROAD NE,402365.36,136678.02,38.897950,-76.972732
1,STANDARD,1004 RHODE ISLAND AVENUE NE,400701.99,139442.62,38.922857,-76.991905
2,STANDARD,2333 FAIRLAWN AVENUE SE,402526.12,134101.82,38.874742,-76.970889
3,STANDARD,720 VARNUM STREET NW,398034.18,141657.91,38.942811,-77.022676
4,STANDARD,700 - 799 BLOCK OF 22ND STREET NW,395763.56,136790.11,38.898952,-77.048838


In [135]:
# Remove City and State
# clean up the Ward to just numbers
# Fix Zipcode to be int.
# Longitude is the same as the X column
# Details can be vectorized.
# What is MARADDRESSREPOSITORYID?
csr_train.iloc[:5, 24:31]

,CITY,STATE,ZIPCODE,MARADDRESSREPOSITORYID,WARD,DETAILS
0,WASHINGTON,DC,20002.0,48983.0,Ward 7,There is some dumping in the rear of this addr...
1,WASHINGTON,DC,20018.0,76304.0,Ward 5,Has not been collected the past 4 weeks.
2,WASHINGTON,DC,20020.0,286919.0,Ward 7,"1 television, 2 vacuums, 1 boom box,"
3,WASHINGTON,DC,20011.0,249794.0,Ward 4,requesting ratb abatement
4,WASHINGTON,DC,20052.0,NaN,2,Broken Parking Meter


##### Check with some categorical columns' values and find typos or misspelled. 

In [136]:
csr_train.PRIORITY.value_counts()

STANDARD     1179004
URGENT         24585
EMERGNCY       23635
EMERGENCY       1189
PRIORITY         141
ESCALATED          1
PRIOR003           1
Name: PRIORITY, dtype: int64

##### Preprocessing

In [13]:
# create preprocess function

def crimespot_preprocess(df):
    # Removing unused or redundent information
    csr_train.drop(['INSPECTORNAME', 'CITY', 'STATE', 'X', 'Y'], axis=1, inplace=True)
    csr_train.columns = map(str.lower, csr_train.columns) # Easier to work with lowercase columns

    # fill nan values to 0
    csr_train.serviceduedate.fillna('0', inplace=True)
    csr_train.resolutiondate.fillna('0', inplace=True)
    csr_train.inspectiondate.fillna('0', inplace=True)
    
    # replace values
    csr_train.priority.replace("EMERGNCY", "EMERGENCY")
    
    # clean up datetime related data
    timestamp = ['adddate', 'resolutiondate', 'serviceduedate', 'serviceorderdate', 'inspectiondate']
    for x in timestamp:
        csr_train[x] = csr_train[x].map(lambda x: x.strip('Z').replace('T', ' '))
    
    # treat zipcode as string and strip
    csr_train.zipcode = csr_train.zipcode.astype(str).str.strip().str.strip('.0')

    # (KS) make single line to combine Brian's on 'Ward'
    csr_train.ward = csr_train.ward.astype(str).map(lambda x: x.strip('Ward')).str.strip().str.strip('.0')

    return df

csr_train = crimespot_preprocess(csr_train)

##### Remainders for the preprocessing (responsible - Kihoon)

In [138]:
csr_train.ward.value_counts()

2      254937
6      214903
4      158945
5      153126
1      127995
7      119438
3      114738
8       80930
nan      6221
Name: ward, dtype: int64

In [ ]:
# (BC)'s code on 6221 missing(nan) values. Maybe we can put in the Ward based on Zipcode

# Since it is no longer, null data. How to fix it? (KS)

# for index, row in csr_train.iterrows():
#     if(pd.isnull(row['WARD'])):
#         row['WARD'] = row['ZIPCODE']
#         #         for x in csr_train:
# #     if pd.isnull(x):
# #         x['WARD'] = csr_train.ZIPCODE


##### New challenge here, team! Thousands of spoiled zipcode data detected .
- Initially, team assumed that we have only 16 null values to manually map zipcode by address
- However, apparently there were tons of spoiled zipcodes. 
- Primarily, Kihoon will take a look on this to fix.

In [139]:
csr_train.zipcode.value_counts()

20002    144190
20011    123232
20001    119199
20019     90170
20009     80421
2002      66351
20007     61568
20003     61316
2001      51723
20016     48526
20032     37641
20018     37447
20008     34014
20005     33947
20017     33281
20015     31462
20037     30120
20036     29633
20024     29490
20012     28990
20006     22403
20004     19572
20052      4942
20415      1011
20059       966
2025        854
2024        636
20405       601
20201       553
20057       523
          ...  
20886         1
2122          1
36            1
-12           1
20613         1
28272         1
20135         1
2051          1
83127         1
-4115         1
23294         1
1             1
-1254         1
16            1
20705         1
22406         1
60435         1
-1865         1
22046         1
20906         1
-2103         1
2077          1
2             1
-2207         1
20746         1
9002          1
-2326         1
20783         1
11            1
24577         1
Name: zipcode, Length: 1

##### Teams, two datasets have different digits in the Lat's and Long's. Will it be matters? (KS)
- Exec decision) keep Lats/Longs for 4 decimal points as 11m errors.

In [5]:
csr_train[['LATITUDE', 'LONGITUDE']].head()

,LATITUDE,LONGITUDE
0,38.897950,-76.972732
1,38.922857,-76.991905
2,38.874742,-76.970889
3,38.942811,-77.022676
4,38.898952,-77.048838


In [6]:
shots_train[['Latitude', 'Longitude']].head()

,Latitude,Longitude
0,38.917,-77.012
1,38.917,-77.002
2,38.917,-76.987
3,38.823,-77.000
4,38.893,-76.993


In [1]:
import geopandas as gpd
from shapely.geometry import Point

In [15]:
geometry = [Point(xy) for xy in zip(csr_train['longitude'], csr_train['latitude'])]
csr_train.drop(['longitude', 'latitude'], axis=1, inplace=True)
crs = {'init': 'epsg:4326'}
gdf = gpd.GeoDataFrame(csr_train, geometry=geometry, crs=crs)

In [16]:
psa = gpd.read_file('./assets/Police_Service_Areas.geojson')

In [17]:
psa = psa[['PSA','geometry']]

In [19]:
csr_train = gpd.sjoin(gdf, psa, how='left', op='within')

In [20]:
csr_train.head()

,objectid,servicecode,servicecodedescription,servicetypecodedescription,organizationacronym,servicecallcount,adddate,resolutiondate,serviceduedate,serviceorderdate,...,streetaddress,xcoord,ycoord,zipcode,maraddressrepositoryid,ward,details,geometry,index_right,PSA
0,463232,S0011,Alley Cleaning,Street Cleaning,DPW,1,2014-01-02 13:27:40.000,2014-01-15 07:43:42.000,2014-02-18 13:27:40.000,2014-01-02 13:27:40.000,...,2301 BENNING ROAD NE,402365.36,136678.02,20002,48983.0,7,There is some dumping in the rear of this addr...,POINT (-76.97273246 38.89794972),22.0,507.0
1,463233,S0321,Recycling Collection - Missed,Recycling,DPW,1,2014-01-02 13:46:57.000,2014-01-06 12:39:39.000,2014-01-06 13:46:57.000,2014-01-02 13:46:57.000,...,1004 RHODE ISLAND AVENUE NE,400701.99,139442.62,20018,76304.0,5,Has not been collected the past 4 weeks.,POINT (-76.99190473 38.92285708),42.0,504.0
2,463234,S0031,Bulk Collection,Bulk Collection,DPW,1,2014-01-02 13:57:46.000,2014-01-14 14:29:16.000,2014-01-23 13:57:46.000,2014-01-02 13:57:46.000,...,2333 FAIRLAWN AVENUE SE,402526.12,134101.82,2002,286919.0,7,"1 television, 2 vacuums, 1 boom box,",POINT (-76.97088871 38.87474188),10.0,605.0
3,463235,S0311,Rat Abatement,DOH,DOH,1,2014-01-02 13:43:20.000,0,2014-02-24 13:43:20.000,2014-01-02 13:43:20.000,...,720 VARNUM STREET NW,398034.18,141657.91,20011,249794.0,4,requesting ratb abatement,POINT (-77.02267596 38.94281117),46.0,407.0
4,463236,S0276,Parking Meter Repair,TOA,DDOT,1,2014-01-02 16:00:59.000,2014-01-07 16:33:48.000,2014-01-09 16:00:59.000,2014-01-02 16:00:59.000,...,700 - 799 BLOCK OF 22ND STREET NW,395763.56,136790.11,20052,NaN,2,Broken Parking Meter,POINT (-77.04883778217619 38.8989521053866),23.0,207.0


In [21]:
csr_train.reset_index()

,index,objectid,servicecode,servicecodedescription,servicetypecodedescription,organizationacronym,servicecallcount,adddate,resolutiondate,serviceduedate,...,streetaddress,xcoord,ycoord,zipcode,maraddressrepositoryid,ward,details,geometry,index_right,PSA
0,0,463232,S0011,Alley Cleaning,Street Cleaning,DPW,1,2014-01-02 13:27:40.000,2014-01-15 07:43:42.000,2014-02-18 13:27:40.000,...,2301 BENNING ROAD NE,402365.360000,136678.020000,20002,48983.0,7,There is some dumping in the rear of this addr...,POINT (-76.97273246 38.89794972),22.0,507.0
1,1,463233,S0321,Recycling Collection - Missed,Recycling,DPW,1,2014-01-02 13:46:57.000,2014-01-06 12:39:39.000,2014-01-06 13:46:57.000,...,1004 RHODE ISLAND AVENUE NE,400701.990000,139442.620000,20018,76304.0,5,Has not been collected the past 4 weeks.,POINT (-76.99190473 38.92285708),42.0,504.0
2,2,463234,S0031,Bulk Collection,Bulk Collection,DPW,1,2014-01-02 13:57:46.000,2014-01-14 14:29:16.000,2014-01-23 13:57:46.000,...,2333 FAIRLAWN AVENUE SE,402526.120000,134101.820000,2002,286919.0,7,"1 television, 2 vacuums, 1 boom box,",POINT (-76.97088871 38.87474188),10.0,605.0
3,3,463235,S0311,Rat Abatement,DOH,DOH,1,2014-01-02 13:43:20.000,0,2014-02-24 13:43:20.000,...,720 VARNUM STREET NW,398034.180000,141657.910000,20011,249794.0,4,requesting ratb abatement,POINT (-77.02267596 38.94281117),46.0,407.0
4,4,463236,S0276,Parking Meter Repair,TOA,DDOT,1,2014-01-02 16:00:59.000,2014-01-07 16:33:48.000,2014-01-09 16:00:59.000,...,700 - 799 BLOCK OF 22ND STREET NW,395763.560000,136790.110000,20052,NaN,2,Broken Parking Meter,POINT (-77.04883778217619 38.8989521053866),23.0,207.0
5,5,463237,S0031,Bulk Collection,Bulk Collection,DPW,1,2014-01-02 16:25:59.000,2014-01-15 13:35:29.000,2014-01-23 16:25:59.000,...,664 MADISON STREET NE,400227.450000,143296.080000,20011,847.0,4,"3 mattresses, entertainment center, table, 2 m...",POINT (-76.99737579000001 38.95757042),53.0,406.0
6,6,463238,S0276,Parking Meter Repair,TOA,DDOT,1,2014-01-02 16:14:47.000,2014-01-06 10:33:16.000,2014-01-09 16:14:47.000,...,800 - 899 BLOCK OF 9TH STREET NW,397920.990000,136943.140000,20001,NaN,2,Broken Parking Meter,POINT (-77.0239673421657 38.9003384044116),19.0,102.0
7,7,463239,S05SL,Streetlight Repair Investigation,TOA,DDOT,1,2014-01-02 16:49:52.000,2014-01-07 03:48:06.000,2014-01-06 16:49:52.000,...,4305 FESSENDEN STREET NW,392689.790000,143004.210000,20016,265877.0,3,between 4307 and 4305,POINT (-77.08433844 38.95491079),51.0,202.0
8,8,463240,S0031,Bulk Collection,Bulk Collection,DPW,1,2014-01-02 16:48:08.000,2014-01-14 11:50:46.000,2014-01-23 16:48:08.000,...,1803 29TH STREET SE,402952.800000,133391.940000,2002,150133.0,7,"desk, chair, exercise machine, fence",POINT (-76.96597465000001 38.86834567),9.0,607.0
9,9,463241,S0441,Trash Collection - Missed,Trash Collection,DPW,1,2014-01-02 16:06:49.000,2014-01-06 12:18:00.000,2014-01-06 16:06:49.000,...,4624 5TH STREET NW,398294.670000,142137.740000,20011,247329.0,4,NaN,POINT (-77.01967236999999 38.94713416),46.0,407.0


In [22]:
csr_train.PSA.isnull().sum()

11

In [23]:
csr_train.columns

Index(['objectid', 'servicecode', 'servicecodedescription',
       'servicetypecodedescription', 'organizationacronym', 'servicecallcount',
       'adddate', 'resolutiondate', 'serviceduedate', 'serviceorderdate',
       'inspectionflag', 'inspectiondate', 'serviceorderstatus', 'status_code',
       'servicerequestid', 'priority', 'streetaddress', 'xcoord', 'ycoord',
       'zipcode', 'maraddressrepositoryid', 'ward', 'details', 'geometry',
       'index_right', 'PSA'],
      dtype='object')

In [26]:
csr_train.drop(['index_right', 'geometry', 
                'ward', 'maraddressrepositoryid', 
                'zipcode', 'ycoord', 'xcoord',
                'streetaddress', 'inspectiondate',
                'inspectionflag', 'organizationacronym'
               ], axis=1, inplace=True)

In [29]:
csr_train.set_index('objectid', inplace=True)